In [5]:
with open("flag.enc", "rb") as f:
    encrypted_data = f.read()

# Known PNG header
png_header = bytes([0x89, 0x50, 0x4E, 0x47, 0x0D, 0x0A, 0x1A, 0x0A])

# Extract partial keystream
partial_keystream = bytes([e ^ p for e, p in zip(encrypted_data[:8], png_header)])
print("Keystream bytes:", partial_keystream.hex())


Keystream bytes: e945bb1c4bafae16


In [6]:
bitstream = []
for byte in partial_keystream:
    bits = f"{byte:08b}"
    bitstream.extend([int(b) for b in bits])


In [7]:
ihdr_chunk = bytes([0x49, 0x48, 0x44, 0x52])
more_ciphertext = encrypted_data[8:12]
more_keystream = bytes([a ^ b for a, b in zip(more_ciphertext, ihdr_chunk)])


In [8]:
def berlekamp_massey(sequence):
    n = len(sequence)
    c = [0] * n
    b = [0] * n
    c[0], b[0] = 1, 1
    l, m, d = 0, -1, 0

    for i in range(n):
        d = sequence[i]
        for j in range(1, l+1):
            d ^= c[j] & sequence[i-j]
        if d == 1:
            t = c[:]
            for j in range(i - m, n):
                c[j] ^= b[j - (i - m)]
            if 2 * l <= i:
                l = i + 1 - l
                m = i
                b = t
    return c[:l+1]

In [9]:
lfsr_poly = berlekamp_massey(bitstream)

In [17]:
lfsr_poly[:5]

[1, 1, 1, 1, 0]

In [18]:
len(lfsr_poly)

33

In [10]:
def lfsr(seed, taps, length):
    reg = seed[:]
    output = []
    for _ in range(length):
        output.append(reg[0])
        feedback = 0
        for t in taps:
            feedback ^= reg[t]
        reg = reg[1:] + [feedback]
    return output


In [15]:
seed = bitstream[:len(lfsr_poly) - 1]
taps = [i for i, b in enumerate(lfsr_poly[1:], start=0) if b == 1]


In [ ]:
from bitarray import bitarray
keystream_bytes = bitarray(bitstream).tobytes()


In [ ]:
decrypted = bytes([a ^ b for a, b in zip(encrypted_data, keystream_bytes)])


In [13]:
from bitarray import bitarray

keystream_bits = lfsr(seed=bitstream[:len(lfsr_poly)-1], taps=[i for i, bit in enumerate(lfsr_poly[1:], start=0) if bit], length=len(encrypted_data)*8)
keystream_bytes = bitarray(keystream_bits).tobytes()

decrypted = bytes([a ^ b for a, b in zip(encrypted_data, keystream_bytes)])


In [14]:
with open("flag.png", "wb") as f:
    f.write(decrypted)
